https://www.kaggle.com/code/anonnosingharay/deepfake-video/notebook

https://www.kaggle.com/datasets/sanikatiwarekar/deep-fake-detection-dfd-entire-original-dataset/code?datasetId=5524489&sortBy=voteCount


xceptionnet:

https://github.com/ondyari/FaceForensics/blob/master/classification/network/models.py

https://huggingface.co/blog/prithivMLmods/siglip2-finetune-image-classification

paper:
https://arxiv.org/html/2503.15867v1

# Download

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import userdata
import os

os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')
os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME')

In [ ]:
!kaggle datasets download -d deepfake-detection-challenge

403 Client Error: Forbidden for url: https://www.kaggle.com/api/v1/datasets/metadata/kmasifurrahman/deepfake-detection-challenge


In [ ]:
!kaggle competitions download -c deepfake-detection-challenge

100% 4.12G/4.13G [00:26<00:00, 145MB/s]
100% 4.13G/4.13G [00:26<00:00, 165MB/s]


In [ ]:
#!/usr/bin/env python
""" Downloads FaceForensics++ and Deep Fake Detection public data release
Example usage:
    see -h or https://github.com/ondyari/FaceForensics
"""
# -*- coding: utf-8 -*-
import argparse
import os
import urllib
import urllib.request
import tempfile
import time
import sys
import json
import random
from tqdm import tqdm
from os.path import join


# URLs and filenames
FILELIST_URL = 'misc/filelist.json'
DEEPFEAKES_DETECTION_URL = 'misc/deepfake_detection_filenames.json'
DEEPFAKES_MODEL_NAMES = ['decoder_A.h5', 'decoder_B.h5', 'encoder.h5',]

# Parameters
DATASETS = {
    'original_youtube_videos': 'misc/downloaded_youtube_videos.zip',
    'original_youtube_videos_info': 'misc/downloaded_youtube_videos_info.zip',
    'original': 'original_sequences/youtube',
    'DeepFakeDetection_original': 'original_sequences/actors',
    'Deepfakes': 'manipulated_sequences/Deepfakes',
    'DeepFakeDetection': 'manipulated_sequences/DeepFakeDetection',
    'Face2Face': 'manipulated_sequences/Face2Face',
    'FaceShifter': 'manipulated_sequences/FaceShifter',
    'FaceSwap': 'manipulated_sequences/FaceSwap',
    'NeuralTextures': 'manipulated_sequences/NeuralTextures'
    }
ALL_DATASETS = ['original', 'DeepFakeDetection_original', 'Deepfakes',
                'DeepFakeDetection', 'Face2Face', 'FaceShifter', 'FaceSwap',
                'NeuralTextures']
COMPRESSION = ['raw', 'c23', 'c40']
TYPE = ['videos', 'masks', 'models']
SERVERS = ['EU', 'EU2', 'CA']


def parse_args():
    parser = argparse.ArgumentParser(
        description='Downloads FaceForensics v2 public data release.',
        formatter_class=argparse.ArgumentDefaultsHelpFormatter
    )
    parser.add_argument('output_path', type=str, help='Output directory.')
    parser.add_argument('-d', '--dataset', type=str, default='all',
                        help='Which dataset to download, either pristine or '
                             'manipulated data or the downloaded youtube '
                             'videos.',
                        choices=list(DATASETS.keys()) + ['all']
                        )
    parser.add_argument('-c', '--compression', type=str, default='raw',
                        help='Which compression degree. All videos '
                             'have been generated with h264 with a varying '
                             'codec. Raw (c0) videos are lossless compressed.',
                        choices=COMPRESSION
                        )
    parser.add_argument('-t', '--type', type=str, default='videos',
                        help='Which file type, i.e. videos, masks, for our '
                             'manipulation methods, models, for Deepfakes.',
                        choices=TYPE
                        )
    parser.add_argument('-n', '--num_videos', type=int, default=None,
                        help='Select a number of videos number to '
                             "download if you don't want to download the full"
                             ' dataset.')
    parser.add_argument('--server', type=str, default='EU',
                        help='Server to download the data from. If you '
                             'encounter a slow download speed, consider '
                             'changing the server.',
                        choices=SERVERS
                        )
    args = parser.parse_args()

    # URLs
    server = args.server
    if server == 'EU':
        server_url = 'http://canis.vc.in.tum.de:8100/'
    elif server == 'EU2':
        server_url = 'http://kaldir.vc.in.tum.de/faceforensics/'
    elif server == 'CA':
        server_url = 'http://falas.cmpt.sfu.ca:8100/'
    else:
        raise Exception('Wrong server name. Choices: {}'.format(str(SERVERS)))
    args.tos_url = server_url + 'webpage/FaceForensics_TOS.pdf'
    args.base_url = server_url + 'v3/'
    args.deepfakes_model_url = server_url + 'v3/manipulated_sequences/' + \
                               'Deepfakes/models/'

    return args


def download_files(filenames, base_url, output_path, report_progress=True):
    os.makedirs(output_path, exist_ok=True)
    if report_progress:
        filenames = tqdm(filenames)
    for filename in filenames:
        download_file(base_url + filename, join(output_path, filename))


def reporthook(count, block_size, total_size):
    global start_time
    if count == 0:
        start_time = time.time()
        return
    duration = time.time() - start_time
    progress_size = int(count * block_size)
    speed = int(progress_size / (1024 * duration))
    percent = int(count * block_size * 100 / total_size)
    sys.stdout.write("\rProgress: %d%%, %d MB, %d KB/s, %d seconds passed" %
                     (percent, progress_size / (1024 * 1024), speed, duration))
    sys.stdout.flush()


def download_file(url, out_file, report_progress=False):
    out_dir = os.path.dirname(out_file)
    if not os.path.isfile(out_file):
        fh, out_file_tmp = tempfile.mkstemp(dir=out_dir)
        f = os.fdopen(fh, 'w')
        f.close()
        if report_progress:
            urllib.request.urlretrieve(url, out_file_tmp,
                                       reporthook=reporthook)
        else:
            urllib.request.urlretrieve(url, out_file_tmp)
        os.rename(out_file_tmp, out_file)
    else:
        tqdm.write('WARNING: skipping download of existing file ' + out_file)


def main(args):
    # TOS
    print('By pressing any key to continue you confirm that you have agreed '\
          'to the FaceForensics terms of use as described at:')
    print(args.tos_url)
    print('***')
    print('Press any key to continue, or CTRL-C to exit.')
    _ = input('')

    # Extract arguments
    c_datasets = [args.dataset] if args.dataset != 'all' else ALL_DATASETS
    c_type = args.type
    c_compression = args.compression
    num_videos = args.num_videos
    output_path = args.output_path
    os.makedirs(output_path, exist_ok=True)

    # Check for special dataset cases
    for dataset in c_datasets:
      dataset_path = DATASETS[dataset]
      # Special cases
      if 'original_youtube_videos' in dataset:
          # Here we download the original youtube videos zip file
          print('Downloading original youtube videos.')
          if not 'info' in dataset_path:
              print('Please be patient, this may take a while (~40gb)')
              suffix = ''
          else:
              suffix = 'info'
          download_file(args.base_url + '/' + dataset_path, out_file=join(output_path,'downloaded_videos{}.zip'.format(suffix)),report_progress=True)
          return

        # Else: regular datasets
      print('Downloading {} of dataset "{}"'.format(
          c_type, dataset_path
      ))

      # Get filelists and video lenghts list from server
      if 'DeepFakeDetection' in dataset_path or 'actors' in dataset_path:
        filepaths = json.loads(urllib.request.urlopen(args.base_url + '/' +
              DEEPFEAKES_DETECTION_URL).read().decode("utf-8"))
        if 'actors' in dataset_path:
          filelist = filepaths['actors']
        else:
          filelist = filepaths['DeepFakesDetection']
      elif 'original' in dataset_path:
          # Load filelist from server
          file_pairs = json.loads(urllib.request.urlopen(args.base_url + '/' +
              FILELIST_URL).read().decode("utf-8"))
          filelist = []
          for pair in file_pairs:
            filelist += pair
      else:
          # Load filelist from server
          file_pairs = json.loads(urllib.request.urlopen(args.base_url + '/' +
              FILELIST_URL).read().decode("utf-8"))
          # Get filelist
          filelist = []
          for pair in file_pairs:
              filelist.append('_'.join(pair))
              if c_type != 'models':
                  filelist.append('_'.join(pair[::-1]))
      # Maybe limit number of videos for download
      if num_videos is not None and num_videos > 0:
        print('Downloading the first {} videos'.format(num_videos))
        filelist = filelist[:num_videos]

      # Server and local paths
      dataset_videos_url = args.base_url + '{}/{}/{}/'.format(
          dataset_path, c_compression, c_type)
      dataset_mask_url = args.base_url + '{}/{}/videos/'.format(
          dataset_path, 'masks', c_type)

      if c_type == 'videos':
          dataset_output_path = join(output_path, dataset_path, c_compression,
                                      c_type)
          print('Output path: {}'.format(dataset_output_path))
          filelist = [filename + '.mp4' for filename in filelist]
          download_files(filelist, dataset_videos_url, dataset_output_path)
      elif c_type == 'masks':
          dataset_output_path = join(output_path, dataset_path, c_type,
                                      'videos')
          print('Output path: {}'.format(dataset_output_path))
          if 'original' in dataset:
              if args.dataset != 'all':
                  print('Only videos available for original data. Aborting.')
                  return
              else:
                  print('Only videos available for original data. '
                        'Skipping original.\n')
                  continue
          if 'FaceShifter' in dataset:
              print('Masks not available for FaceShifter. Aborting.')
              return
          filelist = [filename + '.mp4' for filename in filelist]
          download_files(filelist, dataset_mask_url, dataset_output_path)

      # Else: models for deepfakes
      else:
          if dataset != 'Deepfakes' and c_type == 'models':
              print('Models only available for Deepfakes. Aborting')
              return
          dataset_output_path = join(output_path, dataset_path, c_type)
          print('Output path: {}'.format(dataset_output_path))

          # Get Deepfakes models
          for folder in tqdm(filelist):
              folder_filelist = DEEPFAKES_MODEL_NAMES

              # Folder paths
              folder_base_url = args.deepfakes_model_url + folder + '/'
              folder_dataset_output_path = join(dataset_output_path,
                                                folder)
              download_files(folder_filelist, folder_base_url,
                              folder_dataset_output_path,
                              report_progress=False)   # already done




In [ ]:
import sys

sys.argv = [
    'colab_kernel_launcher.py',                 # dummy script name
    '-d', 'Deepfakes',             # dataset
    '-c', 'c23',                   # compression
    '-t', 'models',                # target type
    '-n', '200',                    # number of videos
    '--server', 'EU2',             # server
    '/content/drive/MyDrive/fake_generated_data/face_forensics/'      # output_path (positional)
]
args = parse_args()
main(args)

By pressing any key to continue you confirm that you have agreed to the FaceForensics terms of use as described at:
http://kaldir.vc.in.tum.de/faceforensics/webpage/FaceForensics_TOS.pdf
***
Press any key to continue, or CTRL-C to exit.

Output path: /content/drive/MyDrive/fake_generated_data/face_forensics/manipulated_sequences/Deepfakes/models


 36%|███▌      | 71/200 [1:31:03<3:04:58, 86.03s/it]

In [ ]:
!unzip -q /content/deepfake-detection-challenge.zip

In [ ]:
!cp -r /content/sample_submission.csv /content/drive/MyDrive/fake_videos/dataset

# code

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import cv2
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Dataset paths
REAL_PATH = "/content/drive/MyDrive/fake_generated_data/face_forensics/original/original_sequences/youtube/c23/videos"
FAKE_PATH = "/content/drive/MyDrive/fake_generated_data/face_forensics/manipulated_sequences/FaceShifter/c23/videos"
OUTPUT_FRAME_SIZE = (128, 128)  # Frame dimensions
FRAME_COUNT = 10  # Number of frames to extract per video
MAX_VIDEOS = 700  # Number of videos to process from each category

# Function to extract frames from a video
def extract_frames(video_path, output_size=(128, 128), frame_count=10):
    cap = cv2.VideoCapture(video_path)
    frames = []
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    step = max(total_frames // frame_count, 1)  # Uniform sampling

    for i in range(frame_count):
        cap.set(cv2.CAP_PROP_POS_FRAMES, i * step)
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, output_size)
        frames.append(frame)
    cap.release()
    return np.array(frames)

# Prepare data and labels
data = []
labels = []

# Process real videos
print("Processing real videos...")
real_videos = os.listdir(REAL_PATH)[:MAX_VIDEOS]   # Limit to 300 videos
for video_file in tqdm(real_videos):
    video_path = os.path.join(REAL_PATH, video_file)
    frames = extract_frames(video_path, output_size=OUTPUT_FRAME_SIZE, frame_count=FRAME_COUNT)
    if len(frames) == FRAME_COUNT:  # Ensure correct frame count
        data.append(frames)
        labels.append(0)  # Label 0 for real

# Process fake videos
print("Processing fake videos...")
fake_videos = os.listdir(FAKE_PATH)[:MAX_VIDEOS]  # Limit to 300 videos
for video_file in tqdm(fake_videos):
    video_path = os.path.join(FAKE_PATH, video_file)
    frames = extract_frames(video_path, output_size=OUTPUT_FRAME_SIZE, frame_count=FRAME_COUNT)
    if len(frames) == FRAME_COUNT:
        data.append(frames)
        labels.append(1)  # Label 1 for fake

# Convert to numpy arrays
data = np.array(data)  # Shape: (num_videos, num_frames, 128, 128, 3)
labels = np.array(labels)

# Split into train, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(data, labels, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Normalize data
X_train = X_train / 255.0
X_val = X_val / 255.0
X_test = X_test / 255.0

# Convert labels to categorical one hot encoding (real and fake class)
y_train = to_categorical(y_train, num_classes=2)
y_val = to_categorical(y_val, num_classes=2)
y_test = to_categorical(y_test, num_classes=2)

print(f"Data shapes: Train - {X_train.shape}, Validation - {X_val.shape}, Test - {X_test.shape}")

Processing real videos...


100%|██████████| 200/200 [10:47<00:00,  3.24s/it]


Processing fake videos...


100%|██████████| 200/200 [10:44<00:00,  3.22s/it]


Data shapes: Train - (280, 10, 128, 128, 3), Validation - (60, 10, 128, 128, 3), Test - (60, 10, 128, 128, 3)


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Augment frames
datagen = ImageDataGenerator(
    horizontal_flip=True,
    rotation_range=10,
    zoom_range=0.1,
    brightness_range=[0.8, 1.2]
)

# Function to augment extracted frames
def augment_frames(frames):
    augmented_frames = []
    for frame in frames:
        frame = datagen.random_transform(frame)
        augmented_frames.append(frame)
    return np.array(augmented_frames)

# Augment training data
augmented_data = []
augmented_labels = []

for i in range(len(X_train)):
    augmented_frames = augment_frames(X_train[i])
    augmented_data.append(augmented_frames)
    augmented_labels.append(y_train[i])

# Combine original and augmented data
X_train_augmented = np.concatenate((X_train, np.array(augmented_data)))
y_train_augmented = np.concatenate((y_train, np.array(augmented_labels)))

print(f"Augmented Train Data: {X_train_augmented.shape}")

Augmented Train Data: (560, 10, 128, 128, 3)


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import Xception
from tensorflow.keras.layers import Dense, Flatten, TimeDistributed, LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout

# Define model
def build_improved_model(input_shape=(FRAME_COUNT, 128, 128, 3)):
    model = Sequential([
        TimeDistributed(Xception(weights='imagenet', include_top=False, input_shape=(128, 128, 3))),
        TimeDistributed(Flatten()),
        Dropout(0.5),  # Add dropout for regularization
        LSTM(128, return_sequences=False),
        Dropout(0.5),  # Add dropout
        Dense(64, activation='relu'),
        Dense(2, activation='softmax')
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

model = build_improved_model()
model.summary()

83683744/83683744 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ time_distributed                │ ?                      │   0 (unbuilt) │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_1              │ ?                      │   0 (unbuilt) │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 20,861,480 (79.58 MB)

 Trainable params: 20,806,952 (79.37 MB)

 Non-trainable params: 54,528 (213.00 KB)

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

# Model checkpoint to save the best model in .keras format
checkpoint = ModelCheckpoint(
    "deepfake_detection_model.keras",  # Change to .keras
    monitor="val_accuracy",
    save_best_only=True,
    verbose=1
)

# Reduce learning rate on plateau
lr_scheduler = ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.5,
    patience=3,
    verbose=1
)

# Train the model
history = model.fit(
    X_train_augmented, y_train_augmented,
    validation_data=(X_val, y_val),
    epochs=40,
    batch_size=10,
    callbacks=[checkpoint, lr_scheduler]
)
model.save("/content/drive/MyDrive/fake_generated_data/deepfake_detection_model_lstm.keras")

Epoch 1/40
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 782ms/step - accuracy: 0.5141 - loss: 0.7538
Epoch 1: val_accuracy improved from -inf to 0.48333, saving model to deepfake_detection_model.keras
56/56 ━━━━━━━━━━━━━━━━━━━━ 167s 1s/step - accuracy: 0.5139 - loss: 0.7536 - val_accuracy: 0.4833 - val_loss: 0.6957 - learning_rate: 1.0000e-04
Epoch 2/40
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 812ms/step - accuracy: 0.5159 - loss: 0.7121
Epoch 2: val_accuracy did not improve from 0.48333
56/56 ━━━━━━━━━━━━━━━━━━━━ 62s 836ms/step - accuracy: 0.5159 - loss: 0.7119 - val_accuracy: 0.3167 - val_loss: 0.6997 - learning_rate: 1.0000e-04
Epoch 3/40
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 805ms/step - accuracy: 0.5262 - loss: 0.6919
Epoch 3: val_accuracy did not improve from 0.48333
56/56 ━━━━━━━━━━━━━━━━━━━━ 82s 829ms/step - accuracy: 0.5256 - loss: 0.6921 - val_accuracy: 0.3667 - val_loss: 0.7015 - learning_rate: 1.0000e-04
Epoch 4/40
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 799ms/step - accuracy: 0.5026 - loss: 0.7089
Epoch 4: val_accur

In [ ]:
from sklearn.metrics import classification_report, accuracy_score

# Load the best saved model
from tensorflow.keras.models import load_model
model = load_model('deepfake_detection_model.keras')

# Evaluate on test set
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)

# Metrics
accuracy = accuracy_score(y_true, y_pred_classes)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

# Precision, Recall, F1-Score
print("Classification Report:")
print(classification_report(y_true, y_pred_classes, target_names=['REAL', 'FAKE']))

2/2 ━━━━━━━━━━━━━━━━━━━━ 25s 12s/step
Test Accuracy: 46.67%
Classification Report:
              precision    recall  f1-score   support

        REAL       0.51      0.67      0.58        33
        FAKE       0.35      0.22      0.27        27

    accuracy                           0.47        60
   macro avg       0.43      0.44      0.43        60
weighted avg       0.44      0.47      0.44        60



face/f++ dataset
Test Accuracy: 46.67%
Classification Report:
              precision    recall  f1-score   support

        REAL       0.52      0.33      0.41        33
        FAKE       0.44      0.63      0.52        27

    accuracy                           0.47        60
   macro avg       0.48      0.48      0.46        60
weighted avg       0.48      0.47      0.46        60


face forensics:

Test Accuracy: 46.67%
Classification Report:
              precision    recall  f1-score   support

        REAL       0.51      0.67      0.58        33
        FAKE       0.35      0.22      0.27        27

    accuracy                           0.47        60
   macro avg       0.43      0.44      0.43        60
weighted avg       0.44      0.47      0.44        60


siglip:

Test Accuracy: 45.00%
Classification Report:
              precision    recall  f1-score   support

        REAL       0.00      0.00      0.00        33
        FAKE       0.45      1.00      0.62        27

    accuracy                           0.45        60
   macro avg       0.23      0.50      0.31        60
weighted avg       0.20      0.45      0.28        60

siglip-fine tune acc = 0.60 on batch 32 epoch 6

In [ ]:
loaded_model = load_model('deepfake_detection_model.keras')

In [ ]:
from tensorflow.keras.models import load_model

# Load the model for real-time detection


def predict_video(video_path, model, output_size=(128, 128), frame_count=10):
    frames = extract_frames(video_path, output_size, frame_count)
    frames = frames / 255.0  # Normalize
    frames = np.expand_dims(frames, axis=0)  # Add batch dimension
    prediction = model.predict(frames)
    label = "FAKE" if np.argmax(prediction) == 1 else "REAL"
    confidence = prediction[0][np.argmax(prediction)]
    print(f"Prediction: {label} (Confidence: {confidence:.2f})")

REAL_PATH = "/content/drive/MyDrive/fake_videos/out1.mp4"
FAKE_PATH = "/kaggle/input/deep-fake-detection-dfd-entire-original-dataset/DFD_manipulated_sequences/DFD_manipulated_sequences"
# Test prediction on a video
# real_sample_path = os.path.join(REAL_PATH, "/kaggle/input/deepfake-testing-videos/model1.mp4")  # Replace with real video path
# fake_sample_path = os.path.join(FAKE_PATH, "/kaggle/input/deepfake-testing-videos/modeloutput1.mp4")  # Replace with fake video path

real_sample_path = "/content/drive/MyDrive/fake_videos/smile_blink.mp4"
fake_sample_path = "/content/drive/MyDrive/fake_videos/spoofed_samiul.mp4"
print("Fake Video Prediction:")
predict_video(fake_sample_path,loaded_model)

print("Real Video Prediction:")
predict_video(real_sample_path,loaded_model)

Fake Video Prediction:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step
Prediction: FAKE (Confidence: 0.67)
Real Video Prediction:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step
Prediction: FAKE (Confidence: 0.74)
